In [ ]:
import json
import regex
import random

from pathlib import Path
from llama_cpp import Llama
from statistics import mean
from datasets import load_dataset
from huggingface_hub import hf_hub_download

from Libraries import Common_Utils as UTL

In [ ]:
HISTORIES = Path("Data/Histories.json")
CONFIG_PATH = Path("Config/config.json")

# Tự động tạo thư mục Data để lưu lịch sử
HISTORIES.parent.mkdir(parents=True, exist_ok=True)
print(f"Thư mục lưu lịch sử: {HISTORIES.parent.resolve()}")
print(f"Đường dẫn file config: {CONFIG_PATH.resolve()}")

# --- 2. Khai báo các biến toàn cục ---
REASON_CLIENT = None
CRITIC_CLIENT = None
HF_DATASET = None
REASON_PROMPT = ""
CRITIC_PROMPT = ""
CONFIG = {}
GENERATION_PARAMS = {}
LLAMA_CPP_PARAMS = {}
FLOW_PARAMS = {}

# --- 3. Tải file cấu hình chính ---
print(f"Đang tải config từ: {CONFIG_PATH}")
CONFIG = UTL.read_json(CONFIG_PATH)

# --- 4. Tải các file Prompt và lấy Cấu hình dựa trên config ---
if CONFIG:
    PROMPT_REASON_PATH = Path(CONFIG["paths"]["prompt_reason"])
    PROMPT_CRITIC_PATH = Path(CONFIG["paths"]["prompt_critic"])

    print(f"Đang tải Reason prompt từ: {PROMPT_REASON_PATH}")
    REASON_PROMPT = UTL.read_text(PROMPT_REASON_PATH)
    
    print(f"Đang tải Critic prompt từ: {PROMPT_CRITIC_PATH}")
    CRITIC_PROMPT = UTL.read_text(PROMPT_CRITIC_PATH)
    
    GENERATION_PARAMS = CONFIG.get("generation_params", {})
    LLAMA_CPP_PARAMS = CONFIG.get("llama_cpp_params", {})
    FLOW_PARAMS = CONFIG.get("flow_params", {})

# --- 5. Kiểm tra kết quả tải file ---
match (bool(CONFIG), bool(REASON_PROMPT), bool(CRITIC_PROMPT)):
    case (False, _, _):
        print("⛔ Dừng chương trình do không tải được file config.")
        raise FileNotFoundError(f"Không tìm thấy hoặc không đọc được file config tại {CONFIG_PATH.resolve()}")
    case (_, False, _) | (_, _, False):
        print("⛔ Dừng chương trình do không tải được file prompt.")
        raise FileNotFoundError(f"Không tìm thấy hoặc không đọc được file prompt (Reason: {PROMPT_REASON_PATH.resolve()} hoặc Critic: {PROMPT_CRITIC_PATH.resolve()})")
    case _:
        print("✅ Tất cả file cấu hình và prompt đã tải thành công.")
        print(f"Cấu hình Llama.cpp sẽ dùng: {LLAMA_CPP_PARAMS}")
        print(f"Cấu hình Flow sẽ dùng: {FLOW_PARAMS}")

### HELPERS

In [ ]:
def initialize_dataset():
    """Tải dataset từ Hugging Face."""
    global HF_DATASET
    DATASET_NAME = CONFIG["dataset"]["name"]
    DATASET_SPLIT = CONFIG["dataset"].get("split", "train")
    
    print(f"📘 Đang tải dataset '{DATASET_NAME}' (split: {DATASET_SPLIT})...")
    try:
        HF_DATASET = load_dataset(DATASET_NAME, split=DATASET_SPLIT)
        print(f"✅ Tải dataset thành công. (Số lượng: {len(HF_DATASET)} mẫu)")
    except Exception as e:
        print(f"❌ Lỗi khi tải dataset: {e}")

In [ ]:
def jsonParse(text: str):
    text = text.strip()
    try:
        return json.loads(text)
    except Exception:
        match = regex.search(r"\{(?:[^{}]|(?R))*\}", text, flags=regex.DOTALL)
        if match:
            json_str = match.group(0)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError:
                pass
        raise ValueError(f"❌ Không parse được JSON từ phản hồi:\n{text}")
    
def randomContent() -> str:
    if HF_DATASET is None:
        print("❌ Lỗi: Dataset chưa được tải. (HF_DATASET is None)")
        return None
    try:
        idx = random.randint(0, len(HF_DATASET) - 1)
        sample = HF_DATASET[idx]
        content = sample.get("article") or sample.get("text")
        title = sample.get("headlines", "Không có tiêu đề")
        if not content:
            print(f"⚠️ Mẫu ngẫu nhiên (index {idx}) không có cột 'article' hoặc 'text', thử lại...")
            return randomContent()
        print(f"📘 Chọn ngẫu nhiên mục: {title}")
        return content
    except Exception as e:
        print(f"❌ Lỗi khi lấy mẫu ngẫu nhiên từ dataset: {e}")
        return None

def averageScore(critical_output: dict) -> float:
    """Tính điểm trung bình từ output JSON lồng nhau của critical model."""
    scoring_dict = critical_output.get("scoring")
    if not scoring_dict or not isinstance(scoring_dict, dict):
        if "error" not in critical_output:
             print("⚠️ Lỗi: Không tìm thấy key 'scoring' trong output của Critical.")
        return 0.0
    keys = ["factuality", "clarity", "logical_coherence", "coverage", "utility", "consistency"]
    vals = [scoring_dict[k] for k in keys if isinstance(scoring_dict.get(k), (int, float))]
    return mean(vals) if vals else 0.0

In [ ]:
def initialize_llm_clients():
    """
    Tải về và khởi tạo cả hai mô hình LLM (Reasoning và Critical) từ Hugging Face Hub.
    """
    global REASON_CLIENT, CRITIC_CLIENT
    
    try:
        # Lấy thông số chung cho LlamaCpp
        llama_params = CONFIG.get("llama_cpp_params", {})
        print(f"Sử dụng Llama params: {llama_params}")
        
        # --- 1. Khởi tạo REASONING MODEL (Qwen 1.8B) ---
        print("\n--- 1. Đang tải Reasoning Model (Qwen 1.8B) ---")
        reason_config = CONFIG["models"]["reasoning_model"]
        
        print(f"Downloading {reason_config['hf_filename']} từ {reason_config['hf_repo_id']}...")
        reason_model_path = hf_hub_download(
            repo_id=reason_config["hf_repo_id"],
            filename=reason_config["hf_filename"]
        )
        print(f"Đã tải xong model Reasoning, đường dẫn: {reason_model_path}")
        
        REASON_CLIENT = Llama(
            model_path=reason_model_path,
            **llama_params
        )
        print("✅ Reasoning Model (Qwen) đã được tải thành công.")

        # --- 2. Khởi tạo CRITICAL MODEL (Phi-3 Mini) ---
        print("\n--- 2. Đang tải Critical Model (Phi-3 Mini) ---")
        critic_config = CONFIG["models"]["critical_model"]
        
        print(f"Downloading {critic_config['hf_filename']} từ {critic_config['hf_repo_id']}...")
        critic_model_path = hf_hub_download(
            repo_id=critic_config["hf_repo_id"],
            filename=critic_config["hf_filename"]
        )
        print(f"Đã tải xong model Critical, đường dẫn: {critic_model_path}")
        
        CRITIC_CLIENT = Llama(
            model_path=critic_model_path,
            **llama_params
        )
        print("✅ Critical Model (Phi-3) đã được tải thành công.")
        print("\n--- Tất cả các mô hình đã sẵn sàng! ---")

    except Exception as e:
        print(f"❌ Đã xảy ra lỗi nghiêm trọng khi khởi tạo LLM: {e}")
        REASON_CLIENT = None
        CRITIC_CLIENT = None

### RUNNER

In [ ]:
def reasoningRun(text: str, feedback: str | None = None) -> str:
    """
    Chạy reasoning (với model Phi-2). 
    Sử dụng định dạng prompt 'Instruct/Output' đặc thù của Phi-2.
    """
    
    # 1. Kiểm tra xem REASON_CLIENT (Phi-2) đã được tải chưa
    if REASON_CLIENT is None:
        print("❌ Lỗi: reasoningRun không thể chạy vì REASON_CLIENT (Phi-2) chưa được tải.")
        return ""
        
    # 2. Xây dựng prompt
    if feedback:
        # Chế độ refine: Yêu cầu Phi-2 làm lại dựa trên feedback
        prompt_content = (
            f"Bạn đã tạo ra một bản tóm tắt trước đó. "
            f"Bản tóm tắt đó đã được đánh giá với phản hồi sau:\n"
            f"---BEGIN FEEDBACK---\n{feedback}\n---END FEEDBACK---\n\n"
            f"Vui lòng tạo lại reasoning trace và tóm tắt, có tính đến phản hồi này. "
            f"Sử dụng lại văn bản gốc dưới đây.\n\n"
            f"Văn bản gốc:\n{text}"
        )
    else:
        # Lần chạy đầu tiên: Sử dụng REASON_PROMPT
        prompt_content = f"{REASON_PROMPT}\n\n{text}"
    
    # --- THAY ĐỔI QUAN TRỌNG ---
    # Áp dụng định dạng 'Instruct/Output' của Phi-2
    prompt = f"Instruct: {prompt_content}\nOutput:"
    # --- KẾT THÚC THAY ĐỔI ---
    
    # print(f"--- PROMPT (REASONING - Phi-2) ---\n{prompt}\n---------------------------")

    # 3. Gọi mô hình REASON_CLIENT (Phi-2)
    try:
        completion = REASON_CLIENT(
            prompt,
            max_tokens=GENERATION_PARAMS.get("max_new_tokens", 512),
            temperature=GENERATION_PARAMS.get("temperature", 0.3),
            top_p=GENERATION_PARAMS.get("top_p", 0.9),
            # Phi-2 không dùng stop token đặc biệt,
            # nhưng chúng ta vẫn giữ các token chung
            stop=["<|end|>", "<|endoftext|>", "</s>", "\nInstruct:"] 
        )
        
        if "choices" in completion and len(completion["choices"]) > 0:
            return completion["choices"][0]["text"].strip()
        else:
            print(f"⚠️ Cảnh báo: Response từ REASON_CLIENT (Phi-2) có cấu trúc lạ: {completion}")
            return str(completion).strip()

    except Exception as e:
        print(f"❌ Lỗi khi chạy reasoningRun (REASON_CLIENT - Phi-2): {e}")
        return ""

In [ ]:
def criticalRun(source_text: str, reasoning_output: str) -> dict:
    """Chạy critical BẰNG MÔ HÌNH LOCAL (Phi-3), trả về dict JSON."""
    
    if CRITIC_CLIENT is None:
        print("❌ Lỗi: criticalRun không thể chạy vì CRITIC_CLIENT (Phi-3) chưa được tải.")
        return {"error": "CRITIC_CLIENT (Phi-3) not initialized"}
    
    user_prompt = f"""
    {CRITIC_PROMPT}

    [Reasoning trace]:
    {reasoning_output}

    [Văn bản gốc]:
    {source_text}
    """
    
    final_prompt = f"<|user|>\n{user_prompt}<|end|>\n<|assistant|>"

    try:
        completion = CRITIC_CLIENT(
            final_prompt,
            max_tokens=1024,
            temperature=0.2,
            top_p=0.9,
            
            stop=["<|end|>", "```", "==="] 
        )

        if "choices" not in completion or len(completion["choices"]) == 0:
            raise ValueError("Mô hình CRITIC_CLIENT (Phi-3) không trả về nội dung.")

        raw_output = completion["choices"][0]["text"].strip()
        if not raw_output:
            raise ValueError("Mô hình CRITIC_CLIENT (Phi-3) trả về phản hồi rỗng.")

        try:
            # Bây giờ raw_output sẽ sạch hơn và chỉ chứa JSON
            parsed_json = jsonParse(raw_output)
            return parsed_json
            
        except ValueError as e:
            print(f"⚠️ Lỗi parse JSON từ mô hình CRITIC_CLIENT (Phi-3): {e}")
            print(f"--- RAW OUTPUT TỪ PHI-3 (CRITICAL) ---")
            print(raw_output)
            print(f"----------------------------------------")
            return {"error": "Failed to parse JSON from critical model", "raw_response": raw_output}

    except Exception as e:
        error_message = f"❌ Lỗi khi gọi CRITIC_CLIENT (critical): {str(e)}"
        print(error_message)
        return {"error": error_message}

### MAIN FUNC

In [ ]:
def mainFlow(source_text: str, 
             max_iters=CONFIG["flow_params"]["max_iters"], 
             min_improve=CONFIG["flow_params"]["min_improve"]):
    
    best_reason, best_score = None, 0
    history = []
    current_feedback = None
    current_reasoning = ""

    for step in range(1, max_iters + 1):
        print(f"\n🔄 Vòng {step} ...")
        
        current_reasoning = reasoningRun(source_text, current_feedback)
        
        artifact_marker = "CV=" 
        if artifact_marker in current_reasoning:
            print(f"⚠️ Phát hiện artifact '{artifact_marker}', đang cắt bỏ...")
            artifact_index = current_reasoning.find(artifact_marker)
            current_reasoning = current_reasoning[:artifact_index].strip()
        
        if not current_reasoning:
            print("⛔ Reasoning trả về rỗng, dừng vòng lặp.")
            break
            
        print(f"\n🔄 Reaoning Result: {current_reasoning}")
        
        critical_output = criticalRun(source_text, current_reasoning)
        
        if critical_output.get("error"):
            print(f"⛔ Lỗi từ Critical: {critical_output['error']}, dừng vòng lặp.")
            break
        
        average_score = averageScore(critical_output)
        
        current_feedback = critical_output.get("feedback_text", "") 

        print(f"📊 Điểm TBC: {average_score:.2f}")
        print(f"📝 Nhận xét (Toàn bộ JSON): {json.dumps(critical_output, ensure_ascii=False, indent=2)}\n")

        history.append({
            "round": step, 
            "score": average_score, 
            "reasoning": current_reasoning, 
            "evaluation": critical_output.get("scoring", {}),
            "feedback": current_feedback
        })
        
        if average_score > best_score + min_improve:
            best_reason, best_score = current_reasoning, average_score
            print(f"📈 Cải thiện tốt, điểm mới: {best_score:.2f}")
            if average_score >= 4.8:
                print("🎉 Đạt điểm cao, dừng sớm.")
                break
        else:
            print("⛔ Không cải thiện đáng kể, dừng.")
            break

    return {"best_reasoning": best_reason, "best_score": best_score, "history": history}

### RUN MAIN

In [ ]:
if __name__ == "__main__":
    
    # --- 1. GỌI CÁC HÀM KHỞI TẠO ---
    # Gọi hàm mới để tải cả 2 model
    initialize_llm_clients() 
    initialize_dataset()
    
    # --- 2. KIỂM TRA TÀI NGUYÊN ---
    # ⬅️ Phải kiểm tra cả REASON_CLIENT và CRITIC_CLIENT
    if REASON_CLIENT is None or CRITIC_CLIENT is None or HF_DATASET is None: 
        print("⛔ Không thể bắt đầu mainFlow (một trong các model hoặc dataset chưa được tải).")
        print("Vui lòng kiểm tra lại lỗi ở trên.")
        # Sử dụng 'pass' thay vì 'exit()' để tránh làm crash kernel Colab
        pass 
    else:
        # --- 3. CHẠY QUY TRÌNH CHÍNH ---
        try:
            inputPara = randomContent()
            if inputPara is None:
                print("⛔ Không tải được nội dung test data, dừng chương trình.")
                pass
            else:
                # Sử dụng biến FLOW_PARAMS toàn cục đã tải ở cell config
                print(f"Bắt đầu mainFlow với các tham số: {FLOW_PARAMS}")
                result = mainFlow(
                    inputPara,
                    max_iters=FLOW_PARAMS.get("max_iters", 3),
                    min_improve=FLOW_PARAMS.get("min_improve", 0.05)
                )

                # --- 4. IN KẾT QUẢ ---
                print("\n" + "="*50)
                print("🎯 KẾT QUẢ CUỐI CÙNG 🎯")
                print("="*50)
                
                final_output = {
                    "best_score": result["best_score"],
                    "best_reasoning": result["best_reasoning"]
                }
                
                print(json.dumps(final_output, ensure_ascii=False, indent=2))
                
                # --- 5. LƯU LỊCH SỬ ---
                # Đảm bảo thư mục 'Data' đã được tạo (từ cell config)
                UTL.insert_json(result["history"], HISTORIES, indent=2)
                print(f"\n📝 Lịch sử chạy đầy đủ đã được lưu vào '{HISTORIES}'")

        except Exception as e:
            print(f"\n❌ Đã xảy ra lỗi không mong muốn trong main: {e}")